# Read wikipedia page for Toronto neighborhood data and perform webscraping to extract the postal data

In [24]:
!pip install BeautifulSoup4
import urllib.request
from bs4 import BeautifulSoup
import csv
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
print(urlpage)

Requirement not upgraded as not directly required: BeautifulSoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


### Using urllib requests API read the webpage and extract the table and read the results

In [25]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
# find results within table
table = soup.find('table', attrs={'class': 'wikitable sortable'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 290


### Main logic is part of the below cell where we read through the cells and extract the data and store them in a data frame.

### we use df.ne to exclude the Borough that is 'Not assigned'.

### we use group by and aggregate with join to group by postal code and add comma seperated values

In [26]:
postal_data = []
COLUMNS = ['PostalCode', 'Borough', 'Neighbourhood']
for row in results:
    cells = row.findAll('td')
    if len(cells) == 0: 
        continue
    postal_data.append([cell.text.strip('\n') for cell in cells])
    
df = pd.DataFrame(postal_data, columns=COLUMNS).reset_index(drop=True)
df = df.loc[df.Borough.ne('Not assigned')].reset_index(drop=True)
#df_new = df.groupby(df['PostalCode']).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)
neighborhoods = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
neighborhoods.shape

(103, 3)

In [28]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
df_llc = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

print('Dataset downloaded and read into a pandas dataframe!')


Dataset downloaded and read into a pandas dataframe!


In [29]:
df_llc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
neighborhoods['Latitude'] = neighborhoods['PostalCode'].map(df_llc.set_index('Postal Code')['Latitude']).fillna(0)
neighborhoods['Longitude'] = neighborhoods['PostalCode'].map(df_llc.set_index('Postal Code')['Longitude']).fillna(0)

In [31]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Filter out the dataframe that contains the word 'Toronoto' and move it to 'neighborhoods' data frame

In [32]:
import pandas as pd # library for data analsysis
toronto_data = df_new[df_new['Borough'].str.contains('Toronto')].reset_index(drop=True)
#neighborhoods = pd.DataFrame(df_new)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.37 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  31.69 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  40.24 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.68 MB/s
vincent-0.4.4- 100% |###################